In [1]:
!pip install torch torchvision transformers scikit-learn  
import torch
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'

True


In [2]:
# imports

import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, Dataset  
from transformers import BertTokenizer, BertForSequenceClassification  
import pandas as pd  
from sklearn.metrics import precision_recall_fscore_support  

In [3]:
class CustomDataset(Dataset):  
    def __init__(self, data):  
        self.input_ids = data['input_ids']  
        self.attention_mask = data['attention_mask']  
        self.label = data['label']  

    def __len__(self):  
        return self.input_ids.size(0)  

    def __getitem__(self, idx):  
        return (  
            self.input_ids[idx],   
            self.attention_mask[idx],   
            self.label[idx],   
        )  


In [4]:
train_data_path = "correct_train.pt"  
val_data_path = "correct_val.pt"  

train_data = torch.load(train_data_path)  
val_data = torch.load(val_data_path)  

train_dataset = CustomDataset(train_data)  
val_dataset = CustomDataset(val_data)  

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  

print("Data loaders created successfully.")  

Data loaders created successfully.


<ipython-input-4-58cc5e8aee9f>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(train_data_path)
<ipython-input-4-58cc5e8aee9f>:5: FutureWarning: You

In [5]:

class BiLSTM(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):  
        super(BiLSTM, self).__init__()  
        self.embedding = nn.Embedding(input_dim, hidden_dim)  
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, bidirectional=True, batch_first=True)  
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  

    def forward(self, input_ids, attention_mask):  
        embedded = self.embedding(input_ids)  
        lstm_out, _ = self.lstm(embedded)  
        masked_out = lstm_out * attention_mask.unsqueeze(2)  
        pooled = torch.mean(masked_out, dim=1)  
        output = self.fc(pooled)  
        return output  

input_dim = 30522  
hidden_dim = 128  
output_dim = 5  

bilstm_model = BiLSTM(input_dim, hidden_dim, output_dim)  
bilstm_criterion = nn.CrossEntropyLoss()  
bilstm_optimizer = optim.Adam(bilstm_model.parameters(), lr=0.001)  

print("BiLSTM model initialized.")  

BiLSTM model initialized.


/home/amogh/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [6]:
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)  
bert_model = bert_model.to(device)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  
bert_criterion = nn.CrossEntropyLoss()  
bert_optimizer = optim.Adam(bert_model.parameters(), lr=2e-5)  

print("BERT model initialized.")  

/home/amogh/anaconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from

BERT model initialized.


In [7]:
def train_model_with_logs(model, criterion, optimizer, loader, epochs, is_bert=False):  
    for epoch in range(epochs):  
        print(f"\nEpoch {epoch + 1}/{epochs} running...")  
        model.train()  
        total_loss = 0  
        print(next(model.parameters()).device)
        


        for batch_idx, (input_ids, attention_mask, labels) in enumerate(loader):  
            optimizer.zero_grad()  
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            if is_bert:  
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)  
                loss = outputs.loss  
            else:  
                outputs = model(input_ids, attention_mask)  
                loss = criterion(outputs, labels)  

            loss.backward()  
            optimizer.step()  
            total_loss += loss.item()  

            if (batch_idx + 1) % 10 == 0 or batch_idx == len(loader) - 1:  
                print(  
                    f"Epoch {epoch + 1}/{epochs}, Batch {batch_idx + 1}/{len(loader)}, "  
                    f"Batch Loss: {loss.item():.4f}, Total Loss: {total_loss:.4f}"  
                )  

        print(f"Epoch {epoch + 1}/{epochs} completed. Total Loss: {total_loss:.4f}")  


In [12]:
def validate_model_with_logs(model, loader, is_bert=False):
    print("\nRunning Validation...")
    model = model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    predictions_list = []
    total_batches = len(loader)

    with torch.no_grad():
        for batch_idx, (input_ids, attention_mask, labels) in enumerate(loader):
            #optimizer.zero_grad()
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            if is_bert:
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
            else:
                outputs = model(input_ids, attention_mask)
                preds = torch.argmax(outputs, dim=1)

            predictions_list.extend(preds.tolist())
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Print progress every 10 batches
            if (batch_idx + 1) % 10== 0 or batch_idx == total_batches - 1:
                print(f"Validation Batch {batch_idx + 1}/{total_batches} completed.")

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")
    return predictions_list

In [9]:
torch.cuda.empty_cache()


In [10]:
print("Training BERT")  
train_model_with_logs(bert_model, bert_criterion, bert_optimizer, train_loader, epochs=10, is_bert=True) 

Training BERT

Epoch 1/10 running...
cuda:0
Epoch 1/10, Batch 10/4401, Batch Loss: 1.6157, Total Loss: 16.2445
Epoch 1/10, Batch 20/4401, Batch Loss: 1.7192, Total Loss: 32.2602
Epoch 1/10, Batch 30/4401, Batch Loss: 1.5841, Total Loss: 47.8309
Epoch 1/10, Batch 40/4401, Batch Loss: 1.4459, Total Loss: 62.5742
Epoch 1/10, Batch 50/4401, Batch Loss: 1.1512, Total Loss: 76.3531
Epoch 1/10, Batch 60/4401, Batch Loss: 1.3776, Total Loss: 90.5508
Epoch 1/10, Batch 70/4401, Batch Loss: 1.3032, Total Loss: 105.4470
Epoch 1/10, Batch 80/4401, Batch Loss: 1.5760, Total Loss: 119.1559
Epoch 1/10, Batch 90/4401, Batch Loss: 1.0302, Total Loss: 132.6608
Epoch 1/10, Batch 100/4401, Batch Loss: 1.4257, Total Loss: 146.6169
Epoch 1/10, Batch 110/4401, Batch Loss: 1.2415, Total Loss: 160.7262
Epoch 1/10, Batch 120/4401, Batch Loss: 1.4874, Total Loss: 174.7321
Epoch 1/10, Batch 130/4401, Batch Loss: 1.6398, Total Loss: 188.0443
Epoch 1/10, Batch 140/4401, Batch Loss: 1.2323, Total Loss: 201.7096
Epoch

Epoch 1/10, Batch 1190/4401, Batch Loss: 1.0013, Total Loss: 1484.8570
Epoch 1/10, Batch 1200/4401, Batch Loss: 1.4868, Total Loss: 1496.5814
Epoch 1/10, Batch 1210/4401, Batch Loss: 1.4716, Total Loss: 1507.0319
Epoch 1/10, Batch 1220/4401, Batch Loss: 1.2261, Total Loss: 1517.9858
Epoch 1/10, Batch 1230/4401, Batch Loss: 1.0721, Total Loss: 1527.7584
Epoch 1/10, Batch 1240/4401, Batch Loss: 1.0476, Total Loss: 1537.6592
Epoch 1/10, Batch 1250/4401, Batch Loss: 0.9525, Total Loss: 1547.4470
Epoch 1/10, Batch 1260/4401, Batch Loss: 1.4010, Total Loss: 1558.3639
Epoch 1/10, Batch 1270/4401, Batch Loss: 0.9802, Total Loss: 1568.1528
Epoch 1/10, Batch 1280/4401, Batch Loss: 0.6150, Total Loss: 1576.4318
Epoch 1/10, Batch 1290/4401, Batch Loss: 1.0026, Total Loss: 1587.9316
Epoch 1/10, Batch 1300/4401, Batch Loss: 0.8457, Total Loss: 1598.0644
Epoch 1/10, Batch 1310/4401, Batch Loss: 0.9844, Total Loss: 1606.6645
Epoch 1/10, Batch 1320/4401, Batch Loss: 0.4530, Total Loss: 1617.1187
Epoch 

Epoch 1/10, Batch 2350/4401, Batch Loss: 0.3675, Total Loss: 2514.3843
Epoch 1/10, Batch 2360/4401, Batch Loss: 0.7728, Total Loss: 2521.9056
Epoch 1/10, Batch 2370/4401, Batch Loss: 0.8106, Total Loss: 2528.1917
Epoch 1/10, Batch 2380/4401, Batch Loss: 0.4088, Total Loss: 2536.1418
Epoch 1/10, Batch 2390/4401, Batch Loss: 1.7901, Total Loss: 2543.5888
Epoch 1/10, Batch 2400/4401, Batch Loss: 0.2591, Total Loss: 2549.6839
Epoch 1/10, Batch 2410/4401, Batch Loss: 0.8099, Total Loss: 2556.9702
Epoch 1/10, Batch 2420/4401, Batch Loss: 0.7973, Total Loss: 2564.6374
Epoch 1/10, Batch 2430/4401, Batch Loss: 0.3274, Total Loss: 2570.5499
Epoch 1/10, Batch 2440/4401, Batch Loss: 0.3555, Total Loss: 2576.8521
Epoch 1/10, Batch 2450/4401, Batch Loss: 0.8902, Total Loss: 2584.9184
Epoch 1/10, Batch 2460/4401, Batch Loss: 0.6247, Total Loss: 2590.1251
Epoch 1/10, Batch 2470/4401, Batch Loss: 0.6851, Total Loss: 2597.2666
Epoch 1/10, Batch 2480/4401, Batch Loss: 0.8467, Total Loss: 2604.7436
Epoch 

Epoch 1/10, Batch 3510/4401, Batch Loss: 0.2160, Total Loss: 3155.2697
Epoch 1/10, Batch 3520/4401, Batch Loss: 0.6482, Total Loss: 3161.6730
Epoch 1/10, Batch 3530/4401, Batch Loss: 0.5626, Total Loss: 3167.2420
Epoch 1/10, Batch 3540/4401, Batch Loss: 0.3590, Total Loss: 3172.2534
Epoch 1/10, Batch 3550/4401, Batch Loss: 0.4169, Total Loss: 3176.6699
Epoch 1/10, Batch 3560/4401, Batch Loss: 0.6284, Total Loss: 3183.0706
Epoch 1/10, Batch 3570/4401, Batch Loss: 0.7763, Total Loss: 3188.5537
Epoch 1/10, Batch 3580/4401, Batch Loss: 0.3783, Total Loss: 3193.3408
Epoch 1/10, Batch 3590/4401, Batch Loss: 0.6844, Total Loss: 3198.4681
Epoch 1/10, Batch 3600/4401, Batch Loss: 0.2418, Total Loss: 3203.1715
Epoch 1/10, Batch 3610/4401, Batch Loss: 0.7544, Total Loss: 3207.8128
Epoch 1/10, Batch 3620/4401, Batch Loss: 0.2714, Total Loss: 3212.8123
Epoch 1/10, Batch 3630/4401, Batch Loss: 0.3796, Total Loss: 3217.0974
Epoch 1/10, Batch 3640/4401, Batch Loss: 1.1580, Total Loss: 3223.0698
Epoch 

Epoch 2/10, Batch 260/4401, Batch Loss: 0.2477, Total Loss: 112.1588
Epoch 2/10, Batch 270/4401, Batch Loss: 0.0643, Total Loss: 116.2274
Epoch 2/10, Batch 280/4401, Batch Loss: 0.4034, Total Loss: 120.5972
Epoch 2/10, Batch 290/4401, Batch Loss: 0.1887, Total Loss: 124.9541
Epoch 2/10, Batch 300/4401, Batch Loss: 0.2368, Total Loss: 127.8217
Epoch 2/10, Batch 310/4401, Batch Loss: 1.0694, Total Loss: 133.3650
Epoch 2/10, Batch 320/4401, Batch Loss: 0.3978, Total Loss: 137.0220
Epoch 2/10, Batch 330/4401, Batch Loss: 0.7110, Total Loss: 140.5694
Epoch 2/10, Batch 340/4401, Batch Loss: 0.5041, Total Loss: 144.1177
Epoch 2/10, Batch 350/4401, Batch Loss: 0.7220, Total Loss: 150.3328
Epoch 2/10, Batch 360/4401, Batch Loss: 0.2578, Total Loss: 154.4793
Epoch 2/10, Batch 370/4401, Batch Loss: 0.6086, Total Loss: 159.2111
Epoch 2/10, Batch 380/4401, Batch Loss: 0.2452, Total Loss: 163.6078
Epoch 2/10, Batch 390/4401, Batch Loss: 0.2513, Total Loss: 168.5643
Epoch 2/10, Batch 400/4401, Batch 

...

Epoch 9/10, Batch 3500/4401, Batch Loss: 0.0673, Total Loss: 815.0632
Epoch 9/10, Batch 3510/4401, Batch Loss: 0.2112, Total Loss: 817.1694
Epoch 9/10, Batch 3520/4401, Batch Loss: 0.2584, Total Loss: 819.5013
Epoch 9/10, Batch 3530/4401, Batch Loss: 0.0512, Total Loss: 821.4305
Epoch 9/10, Batch 3540/4401, Batch Loss: 0.2940, Total Loss: 824.3834
Epoch 9/10, Batch 3550/4401, Batch Loss: 0.0727, Total Loss: 826.8433
Epoch 9/10, Batch 3560/4401, Batch Loss: 0.1861, Total Loss: 828.7506
Epoch 9/10, Batch 3570/4401, Batch Loss: 0.3066, Total Loss: 831.3508
Epoch 9/10, Batch 3580/4401, Batch Loss: 0.6017, Total Loss: 833.7189
Epoch 9/10, Batch 3590/4401, Batch Loss: 0.5515, Total Loss: 837.4603
Epoch 9/10, Batch 3600/4401, Batch Loss: 0.2366, Total Loss: 840.3428
Epoch 9/10, Batch 3610/4401, Batch Loss: 0.3771, Total Loss: 842.2323
Epoch 9/10, Batch 3620/4401, Batch Loss: 0.2635, Total Loss: 844.2911
Epoch 9/10, Batch 3630/4401, Batch Loss: 0.0566, Total Loss: 846.8643
Epoch 9/10, Batch 36

Epoch 10/10, Batch 260/4401, Batch Loss: 0.1653, Total Loss: 59.1267
Epoch 10/10, Batch 270/4401, Batch Loss: 0.1195, Total Loss: 60.7099
Epoch 10/10, Batch 280/4401, Batch Loss: 0.1801, Total Loss: 62.6622
Epoch 10/10, Batch 290/4401, Batch Loss: 0.1021, Total Loss: 64.7747
Epoch 10/10, Batch 300/4401, Batch Loss: 0.2502, Total Loss: 67.2597
Epoch 10/10, Batch 310/4401, Batch Loss: 0.1070, Total Loss: 69.5061
Epoch 10/10, Batch 320/4401, Batch Loss: 0.1092, Total Loss: 71.1709
Epoch 10/10, Batch 330/4401, Batch Loss: 0.4047, Total Loss: 73.7520
Epoch 10/10, Batch 340/4401, Batch Loss: 0.4289, Total Loss: 76.4354
Epoch 10/10, Batch 350/4401, Batch Loss: 0.0591, Total Loss: 78.5963
Epoch 10/10, Batch 360/4401, Batch Loss: 0.3055, Total Loss: 80.1864
Epoch 10/10, Batch 370/4401, Batch Loss: 0.2090, Total Loss: 82.4856
Epoch 10/10, Batch 380/4401, Batch Loss: 0.2227, Total Loss: 84.1772
Epoch 10/10, Batch 390/4401, Batch Loss: 0.4424, Total Loss: 86.6151
Epoch 10/10, Batch 400/4401, Batch

Epoch 10/10, Batch 1430/4401, Batch Loss: 0.1792, Total Loss: 330.2415
Epoch 10/10, Batch 1440/4401, Batch Loss: 0.1322, Total Loss: 332.1977
Epoch 10/10, Batch 1450/4401, Batch Loss: 0.1025, Total Loss: 334.2768
Epoch 10/10, Batch 1460/4401, Batch Loss: 0.1054, Total Loss: 336.0343
Epoch 10/10, Batch 1470/4401, Batch Loss: 0.0821, Total Loss: 338.0347
Epoch 10/10, Batch 1480/4401, Batch Loss: 0.2223, Total Loss: 339.3797
Epoch 10/10, Batch 1490/4401, Batch Loss: 0.1148, Total Loss: 341.0700
Epoch 10/10, Batch 1500/4401, Batch Loss: 0.0968, Total Loss: 344.6944
Epoch 10/10, Batch 1510/4401, Batch Loss: 0.2343, Total Loss: 347.2680
Epoch 10/10, Batch 1520/4401, Batch Loss: 0.2517, Total Loss: 349.6224
Epoch 10/10, Batch 1530/4401, Batch Loss: 0.4280, Total Loss: 352.1408
Epoch 10/10, Batch 1540/4401, Batch Loss: 0.2602, Total Loss: 354.6379
Epoch 10/10, Batch 1550/4401, Batch Loss: 0.0801, Total Loss: 356.7597
Epoch 10/10, Batch 1560/4401, Batch Loss: 0.0865, Total Loss: 359.4519
Epoch 

Epoch 10/10, Batch 2590/4401, Batch Loss: 0.1516, Total Loss: 603.1671
Epoch 10/10, Batch 2600/4401, Batch Loss: 0.3548, Total Loss: 605.5223
Epoch 10/10, Batch 2610/4401, Batch Loss: 0.1600, Total Loss: 607.9139
Epoch 10/10, Batch 2620/4401, Batch Loss: 0.3528, Total Loss: 610.0436
Epoch 10/10, Batch 2630/4401, Batch Loss: 0.4556, Total Loss: 612.5338
Epoch 10/10, Batch 2640/4401, Batch Loss: 0.1822, Total Loss: 614.6340
Epoch 10/10, Batch 2650/4401, Batch Loss: 0.1662, Total Loss: 616.9624
Epoch 10/10, Batch 2660/4401, Batch Loss: 0.0549, Total Loss: 619.1421
Epoch 10/10, Batch 2670/4401, Batch Loss: 0.0730, Total Loss: 621.4479
Epoch 10/10, Batch 2680/4401, Batch Loss: 0.2094, Total Loss: 623.4793
Epoch 10/10, Batch 2690/4401, Batch Loss: 0.4473, Total Loss: 626.0532
Epoch 10/10, Batch 2700/4401, Batch Loss: 0.3214, Total Loss: 627.8452
Epoch 10/10, Batch 2710/4401, Batch Loss: 0.2046, Total Loss: 630.1609
Epoch 10/10, Batch 2720/4401, Batch Loss: 0.1531, Total Loss: 632.7928
Epoch 

Epoch 10/10, Batch 3750/4401, Batch Loss: 0.3366, Total Loss: 878.5667
Epoch 10/10, Batch 3760/4401, Batch Loss: 0.3819, Total Loss: 880.6856
Epoch 10/10, Batch 3770/4401, Batch Loss: 0.4367, Total Loss: 883.7261
Epoch 10/10, Batch 3780/4401, Batch Loss: 0.0375, Total Loss: 886.1279
Epoch 10/10, Batch 3790/4401, Batch Loss: 0.3635, Total Loss: 890.1154
Epoch 10/10, Batch 3800/4401, Batch Loss: 0.3710, Total Loss: 892.8368
Epoch 10/10, Batch 3810/4401, Batch Loss: 0.5131, Total Loss: 895.0605
Epoch 10/10, Batch 3820/4401, Batch Loss: 0.1970, Total Loss: 897.4108
Epoch 10/10, Batch 3830/4401, Batch Loss: 0.2383, Total Loss: 899.4557
Epoch 10/10, Batch 3840/4401, Batch Loss: 0.2000, Total Loss: 900.8695
Epoch 10/10, Batch 3850/4401, Batch Loss: 0.2633, Total Loss: 903.4729
Epoch 10/10, Batch 3860/4401, Batch Loss: 0.2393, Total Loss: 905.2097
Epoch 10/10, Batch 3870/4401, Batch Loss: 0.5569, Total Loss: 907.1315
Epoch 10/10, Batch 3880/4401, Batch Loss: 0.2878, Total Loss: 909.5174
Epoch 

In [13]:
print("Validating BERT")
bert_predictions = validate_model_with_logs(bert_model, val_loader, is_bert=True)


Validating BERT

Running Validation...
Validation Batch 10/1101 completed.
Validation Batch 20/1101 completed.
Validation Batch 30/1101 completed.
Validation Batch 40/1101 completed.
Validation Batch 50/1101 completed.
Validation Batch 60/1101 completed.
Validation Batch 70/1101 completed.
Validation Batch 80/1101 completed.
Validation Batch 90/1101 completed.
Validation Batch 100/1101 completed.
Validation Batch 110/1101 completed.
Validation Batch 120/1101 completed.
Validation Batch 130/1101 completed.
Validation Batch 140/1101 completed.
Validation Batch 150/1101 completed.
Validation Batch 160/1101 completed.
Validation Batch 170/1101 completed.
Validation Batch 180/1101 completed.
Validation Batch 190/1101 completed.
Validation Batch 200/1101 completed.
Validation Batch 210/1101 completed.
Validation Batch 220/1101 completed.
Validation Batch 230/1101 completed.
Validation Batch 240/1101 completed.
Validation Batch 250/1101 completed.
Validation Batch 260/1101 completed.
Validati

In [14]:
output_data = {
    "bert_predictions": bert_predictions
}
output_df = pd.DataFrame(output_data)
output_df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv.")

Predictions saved to predictions.csv.


In [15]:
bert_model.save_pretrained("bert_model")
print("Models saved.")

Models saved.
